# Piping Chains and the RunnablePassthrough Class

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
pip show langchain

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Users\Marcus\anaconda3\envs\langchain_env_py312\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [5]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [7]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

In [8]:
# Define chat template objects
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [10]:
# Display chat template variable
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [11]:
# Define chat variable
chat = ChatOpenAI(model_name = 'gpt-4', 
                  seed = 365,
                  temperature = 0,
                  max_tokens = 100)

In [12]:
# Define string output parser
string_parser = StrOutputParser()

In [18]:
# Create LCEL chains piping elements together using the RunnablePassthrough class
chain_tools = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()})
chain_strategy = chat_template_strategy | chat | string_parser

In [19]:
# Print chain tools template invoking input variables
print(chain_tools.invoke({'job title':'data scientist'}))

{'tools': '1. Python\n2. R Programming Language\n3. SQL (Structured Query Language)\n4. Tableau\n5. Jupyter Notebooks'}


In [21]:
# Print chain strategy template invoking input variables
print(chain_strategy.invoke({'tools':'''
1. Python
2. R Programming
3. SQL
4. Tableau
5. Jupyter Notebooks
'''}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Once you have a good understanding of these, move on to more advanced topics like classes and file I/O. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics of R, such


In [22]:
# Combine chains together piping them together
chain_combined = chain_tools | chain_strategy

In [23]:
# Print combined chain invoking input variables
print(chain_combined.invoke({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, operators, control flow, and functions. Once you have a good understanding of these, move on to more advanced topics like classes, exceptions, and modules. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding regularly on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics


In [24]:
# Can combine the two chain inputs to create the same answer as the combined chain
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | chat | string_parser)

In [25]:
print(chain_long.invoke({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Once you have a good understanding of these, move on to more complex topics like classes and objects. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics of R, such as data
